In [1]:
# 載入需要的套件
import os
import numpy as np 
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
# 設定 data_path
dir_data = './data/'
Train = os.path.join(dir_data, 'train.csv')
Test = os.path.join(dir_data, 'test.csv')

# 讀取檔案
Train_data = pd.read_csv(Train)
Test_data = pd.read_csv(Test)

In [32]:
train_Y = np.log1p(Train_data['total_price'])
ids = Test_data['building_id']

tp = copy.deepcopy(np.log1p(Train_data['total_price']))

Train_data = Train_data.drop(['building_id', 'total_price'] , axis=1)
Test_data = Test_data.drop(['building_id'] , axis=1)

df = pd.concat([Train_data,Test_data])
df.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
0,8,21,18674,4,3,2,6271,2,NaN,NaN,...,21,58,1,157,1,2483,1,6011,1,34.469803
1,8,7,18800,5,1,2,7885,2,NaN,NaN,...,7,28,1,115,1,15872,1,32221,1,40.073573
2,8,7,19289,4,1,2,6028,2,NaN,NaN,...,27,78,1,212,1,15760,1,32228,1,54.462081
3,8,21,20385,24,0,2,18325,0,NaN,81138.889762,...,2,20,1,125,1,2568,1,7271,1,99.628966
4,1,21,20657,2,4,2,6880,2,NaN,NaN,...,2,18,1,47,1,2587,1,7442,1,124.131236


In [33]:
df = df.fillna(-1)
train_num = train_Y.shape[0]

In [34]:
corr = copy.deepcopy(df[:train_num])

corr.loc[:,"total_price"] = tp

corr_tar = corr.corr()['total_price']

print(corr_tar.sort_values(na_position='first'))

I_index_5000             NaN
I_index_10000            NaN
II_index_5000            NaN
II_index_10000           NaN
III_index_5000           NaN
III_index_10000          NaN
IV_index_10000           NaN
V_index_5000             NaN
V_index_10000            NaN
VI_index_5000            NaN
VI_index_10000           NaN
VII_index_5000           NaN
VII_index_10000          NaN
VIII_index_5000          NaN
VIII_index_10000         NaN
IX_index_10000           NaN
X_index_5000             NaN
X_index_10000            NaN
XII_index_5000           NaN
XII_index_10000          NaN
XIV_index_5000           NaN
XIV_index_10000          NaN
highschool_rate    -0.484650
elementary_rate    -0.478804
junior_rate        -0.339117
death_rate         -0.315882
parking_way        -0.246073
divorce_rate       -0.239164
XIII_MIN           -0.229212
XI_MIN             -0.209311
                      ...   
II_5000             0.382586
V_1000              0.384847
III_5000            0.386935
X_5000        

In [35]:
new_cols = [x for x in corr_tar.index if abs(corr_tar[x])>=0.4]
new_cols

['parking_price',
 'lon',
 'doc_rate',
 'master_rate',
 'bachelor_rate',
 'jobschool_rate',
 'highschool_rate',
 'elementary_rate',
 'I_10000',
 'II_10000',
 'III_10000',
 'IV_10000',
 'V_5000',
 'V_10000',
 'VI_10000',
 'VII_5000',
 'VII_10000',
 'VIII_5000',
 'VIII_10000',
 'IX_5000',
 'IX_10000',
 'X_10000',
 'XI_10000',
 'XII_10000',
 'XIII_5000',
 'XIII_10000',
 'total_price']

In [36]:
train_X = df[:train_num]
estimator = RandomForestRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.9463046874429659

In [37]:
estimator.fit(train_X, train_Y)

test_X = df[train_num:]
predict = estimator.predict(test_X)

test_price = np.expm1(predict)

submit = pd.DataFrame(ids)
submit['total_price'] = test_price.astype(np.int32)
submit.to_csv("2019-05-15.csv",index=False)